In [1]:
# this notebook iterate over neuron 4D files (already masked and filtered) 
# and stream-pipeline the defined analysis steps 
# and save the output into one dataframe per neuron with the following columns
# output_cols: timepoints, vol_all, vol_stable, growthR_all, growthR_stable, transient_px_N, transient_px_per ....

In [2]:
data_path = '/Users/aichah/Desktop/TLI/data/test_data/'
save_path = '/Users/aichah/Desktop/TLI/data/test_data/output/'
entry_points = [[0, 27, 234, 181],
                [0, 26, 255, 203],
                [0, 28, 172, 193],
                [0, 36, 178, 184],
                [0, 22, 216, 105],
                [0, 45, 232, 194]] # define the entry point for each neuron in the folder
stab_limit = 5 #(No. of timepoints for stable branches calculation)

if save_path[-1] != '/':
    save_path += '/'

In [3]:
import os
import utils.analysis as analysis
import utils.datautils as datautils
import matplotlib.pyplot as plt
import pandas as pd
import tifffile as tif
import numpy as np
import re
from tqdm import tqdm

In [4]:
files_list = datautils.get_file_names(data_path, group_by='', 
                                        order=True, nested_files=False, 
                                        criteria='tif')

first 5 files
['/Users/aichah/Desktop/TLI/data/test_data/220209_P36_N2_mask_flipV_JESUS.tif', '/Users/aichah/Desktop/TLI/data/test_data/220209_P36_N1_mask_flipV_JESUS_new_mask.tif', '/Users/aichah/Desktop/TLI/data/test_data/211121_P36.5_N3_GFP_JESUS_label.tif', '/Users/aichah/Desktop/TLI/data/test_data/211113_P36_N2_GFP_JESUS_new_mask.tif', '/Users/aichah/Desktop/TLI/data/test_data/211113_P36_N1_mask3_JESUS.tif']


In [5]:
for i, file in tqdm(enumerate(files_list)):
    file_name = os.path.basename(file)
    start_age = re.search('P(.*)_N', file_name)
    start_age = float(start_age.group(1))
    neuron = tif.imread(file)
    neuron[neuron != 0] = 1
    lifetimes = analysis.cal_lifetimes(neuron, save=False, 
                                        save_path=save_path, save_file=file_name, 
                                        xy_pixel=0.076, z_pixel=0.4)
    stable_neuron = analysis.stable_N(lifetimes, stab_limit=stab_limit, save=False, 
                                        save_path=save_path, save_file=file_name, 
                                        xy_pixel=0.076, z_pixel=0.4)
    output_results = analysis.N_volume(neuron, stable_neuron, normalize=False, 
                                        start_t=start_age, plot=False, 
                                        save=False, save_path=save_path, 
                                        save_file='Vol_'+file_name)
    output_results['growthR_all'] = output_results.vol_all.diff()/output_results.vol_all
    output_results['growthR_stable'] = output_results.vol_stable.diff()/output_results.vol_stable
    output_transient = analysis.trans_px_levels(lifetimes, stab_limit=stab_limit, 
                                                start_t=start_age, plot=False, 
                                                save=False, save_path='', 
                                                save_file='')
    output_results = pd.merge(output_results, output_transient, on='timepoints')
    DGIs = analysis.calculate_DGI(entry_points[i], neuron, 
                                  start_t=start_age, 
                                  save=False, save_path='', 
                                  save_file='')
    output_results = pd.merge(output_results, DGIs, on='timepoints')
    output_file = save_path+file_name+'_output.csv'
    output_results.to_csv(output_file, sep=';')

    # del neuron, stable_neuron, lifetimes, DGIs, output_results, output_transient

100%|██████████| 92/92 [00:01<00:00, 47.51it/s]
6it [04:19, 43.23s/it]


IndexError: list index out of range